<a target="_blank" href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/session.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Sessionクラス解説

[api 一覧はこちら](./manual.ipynb)

Session クラスはAgentの各イベント(on_tick, on_clock, on_update)時に引数として送られてきます。

内部に現在時刻 `timestamp`を保持することによりバックテスト、フォーワードテスト、本番どれであっても同一処理のAgent(BOT)が動作するようになっています。

このSessionのインスタンスを利用してAgent内の取引ロジックを開発してください。
使えるプロパティ／メソッドについて説明します。

このマニュアルではSessionオブジェクトを生成してテストしていますが、実際にはユーザがSessionオブジェクトを直接自分で生成することはなく、フレームワーク側で生成しAgentに送られます。

## プロパティ

### 実行状況表すプロパティ

* `timestamp` 現在時刻をUnixtime(us)で保存取得します。
* `execute_mode` 現在の実行モードを返す。`Real`(本番), `Dummy`(バックテスト), `Dry`(フォーワードテスト）のどれかが得られる。

### 市況を示すプロパティ
* `board` Marketの板情報(`bit`, `ask`)でまとめて取得する
* `last_price` 直近の約定履歴から想定される`bit`, `ask`の最良値（`bit`, `ask`)でまとめて取得する。板情報が取れないときに代わりに利用する。

### 自分のオーダー表すプロパティ
* `buy_orders` 板にある未約定買いオーダー（一覧）
* `buy_order_amount` 板にある未約定買いオーダー注文量合計
* `sell_orders` 板にある未約定売りオーダー（一覧）
* `sell_order_amount` 板にある未約定売りオーダー注文量合計

### 仮想ポジション・利益を表すプロパティ
セッションがはじまってからのポジション・利益を取得する。
* `position` 仮想ポジション

### ログクラスを取得するプロパティ
実行終了後、ログを分析するためにセッションからログを取り出すプロパティ

* `log` 実行結果が入ったLoggerクラスのオブジェクトが取得できる。利用方法は[ログクラスマニュアル 参照](./logger.ipynb)

## メソッド

### 市況を得るメソッド

#### ローソク足
使い方
```
session.ohlcv(
    interval=60,    # 足の幅を秒数で指定
    count=5         # 足の本数を指定
)
```

戻り値：　OHLCVの値がPolarsのDataFrame形式で得られる。

---

### オーダーメソッド

#### 成り行き注文(market_order)
使い方
```
session.market_order(
    side='Buy',     # 売り`Buy`, 買い`Sell`を指定
    size=0.01       # オーダーサイズを指定
)
```
戻り値：作成されたオーダーオブジェクト（板の状況にあわせ分割された結果が配列になってきます）


#### 指値注文(limit_order)
使い方
```
session.limit_order(
    side='Buy',     # 売り`Buy`, 買い`Sell`を指定    
    price=40000,    # 指値を指定
    size=0.01       # オーダーサイズを指定
)
```
戻り値：作成されたオーダーオブジェクト（１行のみですが、他のオーダメソッドにあわせ配列形式）

#### キャンセルオーダー(cancel_order)
使い方
```
session.cancel_order(
    order_id='order_id_0001'    #オーダIDを文字列で指定。
)
```

戻り値：キャンセルされたオーダオブジェクト（１行のみですが、他のオーダメソッドにあわせ配列形式）

#### 期限切れ(expire_order)
使い方
```
session.expire_order(
    ttl_sec=60*10       #期限切れにする期間を秒で指定（この例では10分以上前のオーダをキャンセル）
)
```

戻り値(bool)
* `true` 期限切れがオーダーあった場合
* `false`　期限切れオーダーがなかった場合


## 以下テストを兼ねたSessionオブジェクトの内部動作確認

通常ユーザはSessionクラスを生成しません。Agentにおくられてきたオブジェクトを利用するだけですが以下テスト・説明のためSessionクラス単独で生成し動作させています。

In [ ]:
# 必要ライブラリのインストール
! pip install --upgrade pip

# rbotがbot frameworkになります。PyPiに登録されているので、pipでインストールできます。
# まだ開発初期段階なので、インターフェースが変わる可能性があります。
! pip install --upgrade rbot

In [ ]:
# 関連ライブラリのインストール
! pip install pyarrow
! pip install polars
! pip install plotly
! pip install nbformat
! pip install numpy
! pip install pandas
! pip install json2html

In [ ]:
import rbot

# バージョン確認

print("version =", rbot.__version__)

In [ ]:
# 必要ライブラリのインポート

from rbot import Session
from rbot import Runner
from rbot import Binance
from rbot import BinanceConfig
from rbot import ExecuteMode
from rbot import NOW, HHMM, SEC, time_string


from rbot import init_log

In [ ]:
from IPython.display import HTML
from json2html import json2html
import json

In [ ]:
# binance marketオープン

exchange = Binance(production=False)

config = BinanceConfig.BTCUSDT

market = exchange.open_market(config)


market

In [ ]:
# Sessionで動かすbotを作成します。

class DummyAgent:
    def on_init(self, session):
        session.clock_interval_sec = 60     # 60秒ごとにon_clockが呼ばれる
        
    def on_clock(self, session, clock):
        print("現在時刻", session.timestamp, time_string(session.timestamp))
        print("実行モード", session.execute_mode)
    
        if session.execute_mode == "Dry":
            print("Dryモードではいた情報が取れるので表示します")
            bit, ask = session.board
            print("板情報", bit, ask)
            
        bit_edge, ask_edge = session.last_price
        print("最終約定価格", bit_edge, ask_edge)
        print("")        


In [ ]:
runner = Runner()

agent = DummyAgent()

runner.back_test(exchange=exchange, agent=agent, market=market, execute_time=300)   # 300秒間実行

In [ ]:
class SimpleOrderAgent:
    def __init__(self):
        self.index = 0
        
    def on_init(self, session):
        session.clock_interval_sec = 60*10      # 10分ごとに呼び出される
        print("実行モード", session.execute_mode)
        
    def on_clock(self, session, clock):
        print("---------------------------")
        print("現在時刻", session.timestamp, time_string(session.timestamp))
        print("売り注文", session.sell_orders)
        print("買い注文", session.buy_orders)
        print("ポジション", session.position)

        self.index += 1
        
        if self.index == 1:
            print("成り行き注文テスト(売り)")
            buy_order = session.market_order("Buy", 0.001)
            print("buy_order", buy_order)

        elif self.index == 2:
            print("成り行き注文テスト(買い)")
            sell_order= session.market_order("Sell", 0.001)
            print("sell_order", sell_order)
            
            print("指値注文テスト")            
            pass    
        elif self.index == 3:
            print("指値注文テスト")
            bit, ask = session.last_price
            buy_order = session.limit_order("Buy", bit-100, 0.001)
            print("buy_order", buy_order)
            
            sell_order = session.limit_order("Sell", ask+100, 0.001)
            print("sell_order", sell_order)

        elif self.index == 4:
            print("Cancelテスト(SELL)")
            if session.sell_orders:
                order = session.sell_orders[0]
                cancel_order = session.cancel_order(order.order_id)
                print("cancel_order=", cancel_order)

        elif self.index == 5:
            print("Exipreテスト1")
            
            if session.expire_order(60*30): # 30分前の注文をキャンセル
                print("expire_order exist")
            else:
                print("expire_order not exist")

        elif self.index == 6:
            print("Exipreテスト2")
            
            if session.expire_order(60*10): # 10分前の注文をキャンセル
                print("expire_order exist")
            else:
                print("expire_order not exist")
        
        print("---------------------------")        
    
    def on_update(self, session, update):
        print("更新情報", update)

In [ ]:
runner = Runner()

agent = SimpleOrderAgent()

runner.back_test(exchange=exchange, agent=agent, market=market, execute_time=30000)   # 300秒間実行